In [1]:
from database import database
from matplotlib import pyplot as plt
import sklearn.feature_extraction.image
from sklearn.feature_extraction.image import extract_patches
import tables
import random
from SRCNN import run
from build_model import build_model
import tensorflow as tf

In [2]:
random.seed(31415926)
args = {}
args['filedir'] = 'train_data'
args['pattern'] = '*.bmp'
args['database_name'] = 'mock'
args['export_dir'] = 'export'
args['patch_shape'] = (64,64,3)
args['stride_size'] = 32



data_set = database(**args)
data_set.initialize()

In [4]:
run(data_set, num_epoch = 15, batch_size = 13)

Preparing data...


100%|████████████████████████████████████████| 226/226 [00:07<00:00, 28.76it/s]


EPOCH: 0 Avg Loss: 0.27241572014064386


100%|████████████████████████████████████████| 226/226 [00:07<00:00, 29.10it/s]


EPOCH: 1 Avg Loss: 0.26617340245737436


...

EPOCH: 12 Avg Loss: 0.20860840283762827


100%|████████████████████████████████████████| 226/226 [00:08<00:00, 28.19it/s]


EPOCH: 13 Avg Loss: 0.204260389930445


100%|████████████████████████████████████████| 226/226 [00:08<00:00, 27.49it/s]


EPOCH: 14 Avg Loss: 0.2000413836779452


In [6]:
print(data_set.size('train'))

2946


In [7]:
img_size,label_size=data_set.peek('train')
print(img_size,img_size)

(2946, 64, 64, 3) (2946, 64, 64, 3)
